Welcome! This script should generate a brand new spike dataset (numpy arrays) from the .wav home sound dataset. This is to save on memory and time, as we won't have to preprocess the dataset multiple times. Trust me, this script takes a loooooong time to run.

You don't have to run this script, as the dataset is saved to my google drive and I will have zipped it up and shared it with you all.

I took some help from AI to write this code.

In [1]:
# run once, also runtime may ask to be restarted, that's fine
!pip install rockpool
!pip install "numpy<2" "jax==0.4.20" "jaxlib==0.4.20"
!pip install samna
!pip install bitstruct

In [2]:
from zipfile import ZipFile

from google.colab import drive
drive.mount('/content/drive')

zip_file_path = '/content/drive/MyDrive/D7046E_SNN_project_dataset/building_106_kitchen.zip'
extract_path = '/content/dataset'

with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

audio_root_dir = '/content/dataset/building_106_kitchen/training_segments'
output_dir = '/content/drive/MyDrive/D7046E_SNN_project_dataset/kitchen_spike_dataset'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
import librosa
from rockpool.devices.xylo.syns61201 import AFESim
from rockpool.timeseries import TSContinuous
from tqdm import tqdm
import json
import gc


def convert_audio_to_spikes(audio_path, sample_T, raster_period):

    # create a new AFESim for each file
    afe = AFESim(
        fs=110e3,
        raster_period=raster_period,
        max_spike_per_raster_period=15,
        add_noise=False,
        add_offset=False,
        add_mismatch=False,
        seed=None,
    ).timed()

    audio, sr = librosa.load(audio_path, sr=None, mono=True)

    times = np.arange(len(audio)) / sr
    ts_audio = TSContinuous(times, audio[:, None])

    # we have a bunch of garbage collection like this to save memory
    del audio, times

    # convert audio to spikes
    spikes, _, _ = afe(ts_audio)

    del ts_audio, afe

    # convert to raster format (time_steps x channels)
    spike_raster = spikes.raster(dt=raster_period, add_events=True)
    del spikes

    # validate we got spikes
    if spike_raster.shape[0] == 0:
        # if no spikes generated, create empty raster
        spike_raster = np.zeros((sample_T, 16), dtype=np.float32)
        return spike_raster

    # pad or truncate to sample_T time steps
    # (300 10ms time steps = 3 seconds)
    if spike_raster.shape[0] < sample_T:
        padding = np.zeros((sample_T - spike_raster.shape[0], spike_raster.shape[1]), dtype=np.float32)
        spike_raster = np.vstack([spike_raster, padding])
        del padding
    else:
        spike_raster = spike_raster[:sample_T, :]

    spike_raster = spike_raster.astype(np.float32)
    return spike_raster


def preprocess_batch(
    audio_root_dir,
    output_dir,
    sample_T,
    raster_period,
    batch_start=0,
    batch_size=4
):
    os.makedirs(output_dir, exist_ok=True)

    # Get all classes
    all_classes = sorted([d for d in os.listdir(audio_root_dir)
                         if os.path.isdir(os.path.join(audio_root_dir, d))])

    # Select batch of classes to process
    classes_to_process = all_classes[batch_start:batch_start + batch_size]

    if not classes_to_process:
        print(f"No classes to process at batch_start={batch_start}")
        return

    print(f"Processing batch: classes {batch_start} to {batch_start + len(classes_to_process) - 1}")
    print(f"Classes in this batch: {classes_to_process}")

    # Create/load metadata
    metadata_path = os.path.join(output_dir, 'metadata.json')
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        class_to_idx = metadata['class_to_idx']
    else:
        # first batch - create metadata
        class_to_idx = {cls: i for i, cls in enumerate(all_classes)}
        metadata = {
            'classes': all_classes,
            'class_to_idx': class_to_idx,
            'sample_T': sample_T,
            'raster_period': raster_period,
            'num_channels': 16,
        }
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"Created metadata for all {len(all_classes)} classes")

    # load existing sample info if it exists
    sample_info_path = os.path.join(output_dir, 'sample_info.npy')
    if os.path.exists(sample_info_path):
        sample_info = np.load(sample_info_path, allow_pickle=True).tolist()
    else:
        sample_info = []

    # Process each class in this batch
    total_processed = 0
    total_errors = 0
    error_log = []

    for class_name in classes_to_process:
        class_path = os.path.join(audio_root_dir, class_name)
        class_idx = class_to_idx[class_name]

        # Create output directory for this class
        class_output_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        # Get all WAV files
        wav_files = [f for f in os.listdir(class_path) if f.endswith('.wav')]

        print(f"\nProcessing '{class_name}' ({len(wav_files)} samples)...")

        class_errors = 0
        for idx, wav_file in enumerate(tqdm(wav_files, desc=f"  {class_name}")):
            audio_path = os.path.join(class_path, wav_file)
            output_filename = wav_file.replace('.wav', '.npy')
            output_path = os.path.join(class_output_dir, output_filename)

            # Skip if already processed
            if os.path.exists(output_path):
                continue

            spike_raster = convert_audio_to_spikes(
                audio_path, sample_T, raster_period
            )
            np.save(output_path, spike_raster)
            sample_info.append((output_path, class_idx))
            total_processed += 1
            del spike_raster

            if idx % 5 == 0:
                gc.collect()


        # Save sample info after each class
        np.save(sample_info_path, sample_info, allow_pickle=True)
        gc.collect()


    return len(all_classes), batch_start + batch_size



# Clear memory
gc.collect()

# AFESim parameters (not creating it here - created fresh for each file)
sample_T = 300
raster_period = 10e-3

# PROCESS IN BATCHES (of classes)
batch_start = 23  # start at class

batch_size = 1  # process n classes at a time

print(f"\nConfiguration:")
print(f"  Batch start: {batch_start}")
print(f"  Batch size: {batch_size}")

total_classes, next_batch = preprocess_batch(
    audio_root_dir=audio_root_dir,
    output_dir=output_dir,
    sample_T=sample_T,
    raster_period=raster_period,
    batch_start=batch_start,
    batch_size=batch_size
)



Configuration:
  Batch start: 23
  Batch size: 1
Processing batch: classes 23 to 23
Classes in this batch: ['water_tap']

Processing 'water_tap' (115 samples)...


  water_tap: 100%|██████████| 115/115 [06:18<00:00,  3.29s/it]


Becuase TimedModuleWrapper was giving me trouble I tried switching to non-timed simulation.

First 4 classes have 10 ms raster period, max 15 spikes: 88.9% empty\
First 4 classes have 20 ms raster period, max 100 spikes: 91.7% empty\
First 4 classes have 50 ms raster period, max 100 spikes: 91.7% empty\
First 4 classes have 5 ms raster period, max 100 spikes: 91.7% empty

conclusion: adjusting these parameters doesn't help make more spikes

Tried scaling voltage of input to expected RMS:\
First 4 classes have 10 ms raster period, max 15 spikes: 11.1% empty (didn't end up using this anyway, while fewer samples are empty, most are still super sparse, this is still worth considering though)

In the end, I switched back to TimedModuleWrapper (.timed()) and created a new afesim for every single audio sample, making sure to delete each simulation and variables after I've used them to save on memory. This is very slow but seems to work fine.
